# Problem loan prediction 2017 - Lending Club
## Code
### For more information about the results of this project, check the report.pdf file

In [ ]:
import pandas as pd
#import datetime as dt

## Uploading the data

The data was dowloaded from the Lending Club website in two files, one for each quarter.

Uploading each file separately and then merging it into a single pandas data frame.

In [ ]:
# Uploading data into two dataframes

data_1Q = pd.read_csv('LoanStats_2017Q1.csv', skiprows=1, low_memory=False)
data_2Q = pd.read_csv('LoanStats3a.csv', skiprows=1, low_memory=False)

# Merging the dataframes

data_loans = pd.concat([data_1Q, data_2Q])

In [ ]:
print (data_loans.shape)
print("HEAD")
display(data_loans.head())
print("\n TAIL")
display(data_loans.tail())

There are a few data descriptors at the end of the files than need to be remove. 

In [ ]:
data_loans = data_loans.sort_values(by = 'id',na_position = 'first', ascending=True)
display(data_loans.tail(10))

In [ ]:
print ('The data frame has %d rows (loans) with %d columns' % (data_loans.shape[0], data_loans.shape[1]))
data_loans.drop(data_loans.tail(5).index,inplace=True)
display(data_loans.tail(5))

Remove the columns with only NaN and or with large number of NaN

In [ ]:
# Create df with number of NaN on each column
use_col = pd.DataFrame(data_loans.isnull().sum().sort_values())
use_col.reset_index(drop = False, inplace=True)
use_col.columns = (['col_name','num_NaN'])
display(use_col)

In [ ]:
pd.DataFrame(data_loans.isnull().sum().sort_values()[data_loans.isnull().sum().sort_values()<10])

In [ ]:
data_loans = data_loans.dropna(axis=1, how='all')
data_loans.shape

In [ ]:
def df_col_low_nan(df_data, min_nan): 
    # Returns a df with columns with less nan than min_nan
    use_col = pd.DataFrame(df_data.isnull().sum().sort_values()[df_data.isnull().sum().sort_values()<min_nan])
    use_col.reset_index(drop = False, inplace=True)
    use_col.columns = (['col_name','num_NaN'])
    new_data = pd.DataFrame()
    for i in range(0,len(use_col.col_name)):
        df = df_data[use_col.col_name[i]]
        new_data = pd.concat([new_data, df], axis=1)
    return new_data


In [ ]:
n_loan_df = df_col_low_nan(data_loans, 10)
n_loan_df.shape

In [ ]:
#data_loans
#data_2017 = pd.read_csv('loans_2017.csv')
n_loan_df.to_csv('loans_2017.csv', index=False)

In [ ]:
n_loan_df = pd.read_csv('loans_2017.csv')

## Cleaning the target feature (loan_status)

For the purpose of this project, problem loans will does that are not paid, 

In [ ]:
n_loan_df.loan_status.unique()

In [ ]:
# Removing fully paid loans
n_loan_df = n_loan_df[n_loan_df.loan_status.str.contains('Current')==False]
n_loan_df.loan_status=n_loan_df.loan_status.str.replace(r'(^.*Paid$)', "Fully Paid")
n_loan_df.loan_status=n_loan_df.loan_status.str.replace(r'(^.*Charged Off$)', "Charged Off")
n_loan_df.loan_status = n_loan_df.loan_status.str.replace(r'(^Late.*$)', "Late")

n_loan_df.loan_status.unique()

In [ ]:
n_loan_df.describe()

In [ ]:
n_loan_df.dtypes.sort_values()

In [ ]:
def object_columns(df):
    non_num_cols = df.dtypes[df.dtypes == 'object']
    return non_num_cols
object_columns(n_loan_df)

In [ ]:
def object_checker (df1, df2, num):
    # df1 dataframe whit the columns
    # df2 dataframe with colum names
    for col in range(0,len(df2)):
        col = df2.index[col]
        values = df1[col].unique()
        if len(values) > num:
           print("\n", col, "\n",len(values),"\n" ,values)
        #print (len(values))
        #print("\n", col, "\n",len(values),"\n" ,values)
object_checker(n_loan_df, object_columns(n_loan_df), 10)

In [ ]:
# Converting int_rate to number
n_loan_df.int_rate = n_loan_df.int_rate.str.replace('%', " ").astype(float)

# Dropping zip_code as there is a state column 
n_loan_df = n_loan_df.drop(["zip_code"], axis=1)

# Removing sub_grade
n_loan_df = n_loan_df.drop(["sub_grade"], axis=1)

# Removing 2110 with n/a in emp_length
n_loan_df = n_loan_df[n_loan_df.emp_length.str.contains('n/a')==False]

# Converting emp_length to number
# less than one year is now 0
n_loan_df.emp_length = n_loan_df.emp_length.str.replace(r'( years)|( year)|(\+ years)', "").str.replace('< 1','0').astype(float)

# Converting date data to number
# issue_d is now time since issued
n_loan_df.issue_d = ((max(pd.to_datetime(n_loan_df['issue_d'])) - pd.to_datetime(n_loan_df['issue_d'])).dt.days/365).astype(int)

# last_credit_pull_d is now time since last credit check
#import datetime as dt
n_loan_df.last_credit_pull_d = (max(pd.to_datetime(n_loan_df['last_credit_pull_d'])) - pd.to_datetime(n_loan_df['last_credit_pull_d'])).dt.days/365
n_loan_df.last_credit_pull_d = n_loan_df[n_loan_df.last_credit_pull_d.isnull() == False]


In [ ]:
object_checker(n_loan_df, object_columns(n_loan_df), 0)

In [ ]:
n_loan_df.dtypes.sort_values()

In [ ]:
#n_loan_df.last_credit_pull_d.sort_values()
n_loan_df[n_loan_df.last_credit_pull_d.isnull() == True]

In [ ]:
len(n_loan_df)

In [ ]:
from dateutil import relativedelta

relativedelta.relativedelta( pd.to_datetime(n_loan_df['issue_d']), max(pd.to_datetime(n_loan_df['issue_d'])) )

In [ ]:
n_loan_df.issue_d = ((max(pd.to_datetime(n_loan_df['issue_d'])) - pd.to_datetime(n_loan_df['issue_d'])).dt.days/365).astype(int)
#.astype('timedelta64[h]')


In [ ]:
n_loan_df['issue_d']

In [ ]:
type(df[:0])

In [ ]:
n_loan_df.last_credit_pull_d = n_loan_df[n_loan_df.last_credit_pull_d.isnull()==False]

# 